In [1]:
import pandas as pd
import numpy as np
from psaw import PushshiftAPI
import psycopg2 as pg
import psycopg2.extras
import datetime

In [2]:
start_time = int(datetime.datetime(2022,5,14).timestamp())

In [3]:
api = PushshiftAPI()

In [4]:
### Create a generator onject with {limit} submissions from {subreddit}
### Showing only fields in {filter}
connection = pg.connect(
   database="postgres", user='postgres', password='admin', host='localhost', port= '5432'
)
cursor = connection.cursor()

cursor.execute("""CREATE EXTENSION IF NOT EXISTS timescaledb CASCADE;""")
cursor.execute("""
        CREATE TABLE IF NOT EXISTS  mention (
        stock_id VARCHAR(255),
        dt TIMESTAMP WITHOUT TIME ZONE NOT NULL,
        message TEXT NOT NULL,
        source TEXT NOT NULL,
        url TEXT NOT NULL,
        PRIMARY KEY (stock_id, dt),
        CONSTRAINT fk_mention_stock FOREIGN KEY (stock_id) REFERENCES tickers (internal_id)
        );
        SELECT create_hypertable('mention', 'dt', if_not_exists => TRUE);
    """)
cursor = connection.cursor(cursor_factory=psycopg2.extras.DictCursor)
cursor.execute("""
    SELECT * FROM tickers
""")
rows = cursor.fetchall()
stocks = {}
for row in rows:
     stocks[f"${row['symbol']}"] = row["internal_id"]
        
submissions = list(api.search_submissions(after= start_time,
                            subreddit = 'wallstreetbets',
                            filter = ["url", "author", "title", "subreddit"],
                            limit = 300
                                         ))
current_source = "WallStreetBets"
for submission in submissions:
    words = submission.title.split()
    cashtags = list(set(filter(lambda word: word.lower().startswith("$"), words)))



    if len(cashtags) > 0:
        print(cashtags)
        #### insert of mention table (place inside loop)
        for cashtag in cashtags:
            submitted_time = datetime.datetime.fromtimestamp(submission.created_utc).isoformat()

            try:
                cursor.execute(f"""
                    INSERT INTO mention (dt, stock_id, message, source, url)
                    VALUES (%s, %s, %s, '{current_source}', %s)
                """,(submitted_time, stocks[cashtag.upper()], submission.title, submission.url))
                connection.commit()
            except Exception as e:
                print(e)
                connection.rollback()

['$MARA']
['$DELL']
['$1.50']
'$1.50'
['$20,000,000,000']
'$20,000,000,000'
['$AMZN']
['$TSLA']
['$362']
'$362'
['$50', '$22.', '$22']
'$50'
'$22.'
'$22'
['$GME']
['$BABA', '$BTO', '$PDD', '$NIO', '$BIDU', '$DLTR', '$DG', '$LULU', '$GME', '$CZR', '$SPLK', '$ETSY']
['$BABA', '$BTO', '$PDD', '$NIO', '$BIDU', '$DLTR', '$DG', '$LULU', '$GME', '$CZR', '$SPLK', '$ETSY']
['$BABA', '$BTO', '$PDD', '$NIO', '$BIDU', '$DLTR', '$DG', '$LULU', '$GME', '$CZR', '$SPLK', '$ETSY']
['$790']
'$790'
['$69']
'$69'
['$GME']
['$']
'$'
['$HMTXF']
['$']
'$'
['$4-$400']
'$4-$400'
['$16']
'$16'
['$4-&gt;$400']
'$4-&GT;$400'
['$CGC']
['$400', '$983']
'$400'
'$983'
['$750']
'$750'
['$PARA', '$GRPN']
['$GME']
['$HEM']
'$HEM'
['$HEM']
'$HEM'
['$5']
'$5'
['$HEM']
'$HEM'
['$rope.']
'$ROPE.'
['$GME']
['$HIBB']
['$1,379', '$8,592']
'$1,379'
'$8,592'


In [5]:
connection.commit()

In [6]:
cursor.execute('SELECT * from pg_extension;')
results = cursor.fetchall()
for result in results:
    print(result)


[13740, 'plpgsql', 10, 11, False, '1.0', None, None]
[16384, 'adminpack', 10, 11, False, '2.1', None, None]
[34336, 'timescaledb', 10, 2200, False, '2.6.1', [34353, 34354, 34375, 34388, 34402, 34401, 34420, 34419, 34435, 34434, 34457, 34473, 34474, 34490, 34502, 34503, 34545, 34552, 34573, 34585, 34595, 34599, 34615, 34634, 34649, 34670, 34676, 34673], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'WHERE id >= 1000', "\r\n  WHERE KEY = 'exported_uuid' ", '', '', '', '', '', '', '', '', '', '', '']]


In [7]:
pd.read_sql("SELECT * FROM mention", connection).columns

C:\Users\thwal\anaconda3\envs\reddit\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Index(['stock_id', 'dt', 'message', 'source', 'url'], dtype='object')

In [8]:
cursor.execute("""
SELECT * FROM mention
""")
rows = cursor.fetchall()
for i in rows:
    print(i)

['b28f4066-5c6d-479b-a2af-85dc1a8f16fb', datetime.datetime(2022, 5, 24, 12, 41, 54), '$spy falling wedge and or downtrend channel Likely to breakout bullish this week from 20% bear territory. Shorts up for a big surprise with that 4M puts purchased today.', 'wallstreetbets', 'https://i.redd.it/fc1ibrddsc191.jpg']
['fc6a5dcd-4a70-4b8d-b64f-d83a6dae9ba4', datetime.datetime(2022, 5, 24, 8, 21, 47), '$FB puts. I’m new at options. Around this price, what will this be worth at market open? $175 Put exp 6/03', 'wallstreetbets', 'https://i.redd.it/151msbpyhb191.jpg']
['83e52ac1-bb18-4e9f-b68d-dda5a8af3ec0', datetime.datetime(2022, 5, 24, 7, 20, 24), '$SNAP who shot ya?', 'wallstreetbets', 'https://i.imgur.com/MlE6nQn.jpg']
['9eae922d-8d67-4c40-b6ea-faca9b092b89', datetime.datetime(2022, 5, 24, 4, 39, 53), 'Minutes before market close my $GME calls had some.. interesting action', 'wallstreetbets', 'https://i.redd.it/yh3an5gdea191.jpg']
['8842d59f-bae0-4bfc-a895-9d97ddedc70b', datetime.datetime(

In [9]:
$

SyntaxError: invalid syntax (1334801704.py, line 1)

In [ ]:
cursor.close()
connection.commit()
connection.close()